In [1]:
using Random
using DataFrames
using CSV
using Statistics

struct Fighter
    name::String
    h::Int16
    f::Int16
    l::Int16
    t::Int16
end

const MXS = 10000
const MXV = 100
const MNV = 10

function random_fighter(h0::Int = MNV, f0::Int = MNV, l0::Int = MNV, t0::Int = MNV)::Fighter
    flag = true
    while flag
        h = h0 + rand(0:(MXV - h0), 1)[1]
        f = f0 + rand(0:(MXV - f0), 1)[1]
        l = l0 + rand(0:(MXV - l0), 1)[1]
        t = t0 + rand(0:(MXV - t0), 1)[1]
        mm = minimum([h, f])
        budget = h*f+h*l+f*t
        if budget <= MXS && budget + mm > MXS
            flag = false
            return Fighter("No Name", h, f, l, t)
        end
    end
end

function eval_battle(a::Fighter, b::Fighter)::Int
    a_finds = a.f >= b.h
    b_finds = b.f >= a.h
    if a_finds && b_finds
        if a.t > b.t
            return 1
        end
        if a.t < b.t
            return -1
        end
        if a.t == b.t
            if a.l > b.l
                return 1
            end
            if a.l < b.l
                return -1
            end
            if a.l == b.l
                if a.h > b.h
                    return 1
                end
                if a.h < b.h
                    return -1
                end
                if a.h == b.h
                    return 0
                end
            end
        end
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        if a.l > b.l
            return 1
        end
        if a.l < b.l
            return -1
        end
        if a.l == b.l
            if a.t > b.t
                return 1
            end
            if a.t < b.t
                return -1
            end
            if a.t == b.t
                if a.f > b.f
                    return 1
                end
                if a.f < b.f
                    return -1
                end
                if a.f == b.f
                    return 0
                end
            end
        end
    end
end

function random_tournament_winner(c::Int)::Fighter
    if c==0
        return random_fighter()
    end
    a = random_tournament_winner(c-1)
    b = random_tournament_winner(c-1)
    res = eval_battle(a, b)
    if res == 1
        return a
    else
        return b
    end
end


tab = CSV.read("census_yob2022_names.txt", DataFrame, header = false)
names = tab.Column1
adjectives = CSV.read("adjectives.csv", DataFrame)
nouns = CSV.read("nouns.csv", DataFrame)

function random_name_and_stat()::Fighter
    vp = [0, 0, 0, 0]
    nametype = rand(1:5)
    name = ""
    if nametype == 1 || nametype == 2
        nm = rand(names)
        adj_i = rand(1:nrow(adjectives))
        adj = adjectives[adj_i, :adjective]
        vp[1] = vp[1] + adjectives[adj_i, :H]
        vp[2] = vp[2] + adjectives[adj_i, :F]
        vp[3] = vp[3] + adjectives[adj_i, :L]
        vp[4] = vp[4] + adjectives[adj_i, :T]
        if nametype == 1
            name = string(nm, " the ", adj)
        end
        if nametype == 2
            name = string(adj, " ", nm)
        end
    end
    if nametype == 3 || nametype == 4
        nm = rand(names)
        noun_i = rand(1:nrow(nouns))
        noun = nouns[noun_i, :noun]
        vp[1] = vp[1] + nouns[noun_i, :H]
        vp[2] = vp[2] + nouns[noun_i, :F]
        vp[3] = vp[3] + nouns[noun_i, :L]
        vp[4] = vp[4] + nouns[noun_i, :T]    
        if nametype == 3
            name = string(nm, " the ", noun)
        end
        if nametype == 4
            name = string(noun, " ", nm)
        end
    end
    if nametype == 5
        adj_i = rand(1:nrow(adjectives))
        adj = adjectives[adj_i, :adjective]
        vp[1] = vp[1] + adjectives[adj_i, :H]
        vp[2] = vp[2] + adjectives[adj_i, :F]
        vp[3] = vp[3] + adjectives[adj_i, :L]
        vp[4] = vp[4] + adjectives[adj_i, :T]
        noun_i = rand(1:nrow(nouns))
        noun = nouns[noun_i, :noun]
        vp[1] = vp[1] + nouns[noun_i, :H]
        vp[2] = vp[2] + nouns[noun_i, :F]
        vp[3] = vp[3] + nouns[noun_i, :L]
        vp[4] = vp[4] + nouns[noun_i, :T]    
        name = string(adj, " ", noun)
    end
    Fighter(name, vp[1], vp[2], vp[3], vp[4])
end

function rand_rename(a::Fighter, n_tries::Int = 10)::Fighter
    best_score = 0.0
    best_b = random_name_and_stat()
    for ii in 1:n_tries
        b = random_name_and_stat()
        b_norm = sqrt(b.h^2 + b.f^2 + b.l^2 + b.t^2)
        score = (a.h * b.h + a.f * b.f + a.l * b.l + a.t * b.t)/b_norm
        if score > best_score
            best_score = score
            best_b = b
        end
    end
    return Fighter(best_b.name, a.h, a.f, a.l, a.t)
end

function eval_battle_list(a::Fighter, bs::Array{Fighter})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function pick_best(as::Array{Fighter}, bs::Array{Fighter})::Fighter
    bestscore = -999
    bestf = as[1]
    for ii in 1:length(as)
        score = eval_battle_list(as[ii], bs)
        if score > bestscore
            bestscore = score
            bestf = as[ii]
        end
    end
    return bestf
end

function pick_best_rdmly(as::Array{Fighter}, bs::Array{Fighter}, ntries::Int)::Fighter
    bestscore = -999
    bestf = rand(as)
    for ii in 1:ntries
        f = rand(as)
        score = eval_battle_list(f, bs)
        if score > bestscore
            bestscore = score
            bestf = f
        end
    end
    return bestf
end

function fighters_to_df(as::Array{Fighter})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ts = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ts[ii] = as[ii].t    
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, t = ts)
    return df
end

function fpart(x::AbstractFloat)::AbstractFloat
  return x - trunc(x)
end

function eval_team_battle(as::Array{Fighter}, bs::Array{Fighter})::Int
    a_i = 1
    b_i = 1
    while (a_i <= length(as)) && (b_i <= length(bs))
        res = eval_battle(as[a_i], bs[b_i])
        if res == 1
            b_i = b_i + 1
        else
            a_i = a_i + 1
            if res == 0
                b_i = b_i + 1
            end
        end
    end
    a_out = (a_i > length(as))
    b_out = (b_i > length(as))
    if a_out
        if b_out
            return 0
        else
            return -1
        end
    else
        return 1
    end
end

eval_team_battle (generic function with 1 method)

In [2]:
function random_team(f::Function, team_size::Int)::Array{Fighter}
    team = Array{Fighter}(undef, team_size)
    for i in 1:team_size
        team[i] = f()
    end
    return team
end

random_team (generic function with 1 method)

In [3]:
library = Array{Fighter}(undef, 100000)
for i in 1:100000
    library[i] = rand_rename(random_fighter())
end

In [4]:
function evaluate_generator(f::Function, team_size::Int, n_times::Int = 1)::AbstractFloat
    scores = Array{AbstractFloat}(undef, n_times)
    for i0 in 1:n_times
        team = Array{Fighter}(undef, team_size)
        for i in 1:team_size
            team[i] = f()
        end
        bestscore = 999999
        exploiter = library[1]
        for i in 1:length(library)
            fighter = library[i]
            score = 0
            for j in 1:team_size
                score = score + eval_battle(team[j], fighter)
            end
            if score < bestscore
                bestscore = score
                exploiter = library[i]
            end
        end
        if n_times < 5
            println(exploiter)        
        end
        scores[i0] = bestscore/team_size
    end
    return mean(scores)
end

evaluate_generator (generic function with 2 methods)

In [5]:
# new random generation to more closely approximate Nash
function random_scheme_a(budget::Int = MXS, h0::Int = MNV, f0::Int = MNV, l0::Int = MNV, t0::Int = MNV)::Fighter
    budget_sofar = budget + 1
    while budget_sofar > budget
        hf_ratio = rand() * .70 + rand() * 0.03 + 0.02
        l_ratio = rand()
        h_temp = rand() + 0.15
        f_temp = rand() + 0.15
        if hf_ratio > 0.05
            while abs(h_temp - f_temp) < 0.3
                h_temp = rand() + 0.15
                f_temp = rand() + 0.15
            end
        end
        budget_hf = (0.05 + hf_ratio * 0.8) * (budget * .75)
        h_ratio = h_temp/sqrt(h_temp * f_temp)
        f_ratio = f_temp/sqrt(h_temp * f_temp)
        h = max(h0, round(sqrt(budget_hf) * h_ratio))
        f = max(f0, round(sqrt(budget_hf) * f_ratio))
        budget_hf = h * f
        budget_remain = (budget - budget_hf) - (l0 * h + t0 * f)
        l_ratio2 = (h^2 + 5 * l_ratio)/(h^2+f^2 + 5)
        #l_ratio2 = (l_ratio * h)/(l_ratio * h + (1-l_ratio) * f)
        lraw = l0 + ( budget_remain * l_ratio2 )/h
        traw = t0 + ( budget_remain * (1 - l_ratio2) )/f
        l = trunc(lraw)
        t = trunc(traw)
        budget_sofar = h * f + h * l + f * t
        if (budget - budget_sofar) >= min(h, f)
            could_add_l = (budget - budget_sofar) >= h
            could_add_t = (budget - budget_sofar) >= f
            if could_add_l && !could_add_t
                l = l+trunc((budget - budget_sofar)/h)
            end
            if !could_add_l && could_add_t
                t = t+trunc((budget - budget_sofar)/f)
            end
            if could_add_l && could_add_t
                if fpart(lraw) > fpart(traw)
                    l = l+trunc((budget - budget_sofar)/h)
                else
                    t = t+trunc((budget - budget_sofar)/f)
                end
            end
        end
    end
    h = convert(Int, h)
    f = convert(Int, f)
    l = convert(Int, l)
    t = convert(Int, t)  
    return Fighter("No Name [Scheme A]", h, f, l, t)
end

#evaluate_generator(random_scheme_a, 1000, 50)

random_scheme_a (generic function with 6 methods)

In [6]:
#evaluate_generator(random_scheme_a, 1000, 50)

In [7]:
budget = MXS
h0 = MNV
f0 = MNV
l0 = MNV
t0 = MNV
hf_ratio0 = 0
l_ratio0 = 0
h_temp0 = 0
f_temp0 = 0

hf_ratio = rand()
l_ratio = rand()
h_temp = rand()
f_temp = rand()
budget_hf = (0.1 + hf_ratio * 0.8) * (budget * .7)
h_ratio = h_temp/sqrt(h_temp * f_temp)
f_ratio = f_temp/sqrt(h_temp * f_temp)
h = max(h0, round(sqrt(budget_hf) * h_ratio))
f = max(f0, round(sqrt(budget_hf) * f_ratio))
budget_hf = h * f
budget_remain = (budget - budget_hf) - (l0 * h + t0 * f)
l_ratio2 = (h-2)^2/((h-2)^2+(f-2)^2)
#l_ratio2 = (l_ratio * h)/(l_ratio * h + (1-l_ratio) * f)
lraw = l0 + ( budget_remain * l_ratio2 )/h
traw = t0 + ( budget_remain * (1 - l_ratio2) )/f
l = trunc(lraw)
t = trunc(traw)
budget_sofar = h * f + h * l + f * t
if (budget - budget_sofar) >= min(h, f)
    could_add_l = (budget - budget_sofar) >= h
    could_add_t = (budget - budget_sofar) >= f
    if could_add_l && !could_add_t
        l = l+trunc((budget - budget_sofar)/h)
    end
    if !could_add_l && could_add_t
        t = t+trunc((budget - budget_sofar)/f)
    end
    if could_add_l && could_add_t
        if fpart(lraw) > fpart(traw)
            l = l+trunc((budget - budget_sofar)/h)
        else
            t = t+trunc((budget - budget_sofar)/f)
        end
    end
end

println((h, f, l, t, budget_hf, budget_sofar, hf_ratio))

(57.0, 87.0, 28.0, 39.0, 4959.0, 9948.0, 0.7599403043844735)


In [8]:
# budget = MXS
# h0 = MNV
# f0 = MNV
# l0 = MNV
# t0 = MNV
# hf_ratio0 = 0
# l_ratio0 = 0
# h_temp0 = 0
# f_temp0 = 0


# for i in 1:1000
#     hf_ratio = rand()
#     l_ratio = 0.25 + 0.5 * rand()
#     h_temp = rand() + 0.1
#     f_temp = rand() + 0.1
#     budget_hf = sqrt(0.05 + hf_ratio * 0.9) * (budget * .7)
#     h_ratio = h_temp/sqrt(h_temp * f_temp)
#     f_ratio = f_temp/sqrt(h_temp * f_temp)
#     h = max(h0, round(sqrt(budget_hf) * h_ratio))
#     f = max(f0, round(sqrt(budget_hf) * f_ratio))
#     budget_hf = h * f
#     budget_remain = (budget - budget_hf) - (l0 * h + t0 * f)
#     lraw = l0 + ( budget_remain * l_ratio )/h
#     traw = t0 + ( budget_remain * (1 - l_ratio) )/f
#     l = trunc(lraw)
#     t = trunc(traw)
#     budget_sofar = h * f + h * l + f * t
#     if (budget - budget_sofar) >= min(h, f)
#         could_add_l = (budget - budget_sofar) >= h
#         could_add_t = (budget - budget_sofar) >= f
#         if could_add_l && !could_add_t
#             l = l+trunc((budget - budget_sofar)/h)
#         end
#         if !could_add_l && could_add_t
#             t = t+trunc((budget - budget_sofar)/f)
#         end
#         if could_add_l && could_add_t
#             if fpart(lraw) > fpart(traw)
#                 l = l+trunc((budget - budget_sofar)/h)
#             else
#                 t = t+trunc((budget - budget_sofar)/f)
#             end
#         end
#     end

#     budget_sofar = h * f + h * l + f * t
#     if (budget - budget_sofar) >= min(h, f)
#         hf_ratio0 = hf_ratio
#         l_ratio0 = l_ratio
#         h_temp0 = h_temp
#         f_temp0 = f_temp
#         println((h, f, l, t, budget_hf, budget_sofar))
#     end
# end

In [9]:
# hf_ratio = hf_ratio0
# l_ratio = l_ratio0
# h_temp = h_temp0
# f_temp = f_temp0

In [10]:
# budget_hf = sqrt(0.05 + hf_ratio * 0.9) * (budget * .7)
# h_ratio = h_temp/sqrt(h_temp * f_temp)
# f_ratio = f_temp/sqrt(h_temp * f_temp)
# h = max(h0, round(sqrt(budget_hf) * h_ratio))
# f = max(f0, round(sqrt(budget_hf) * f_ratio))
# budget_hf = h * f
# budget_remain = (budget - budget_hf) - (l0 * h + t0 * f)
# lraw = l0 + ( budget_remain * l_ratio )/h
# traw = t0 + ( budget_remain * (1 - l_ratio) )/f
# l = trunc(lraw)
# t = trunc(traw)
# budget_sofar = h * f + h * l + f * t
# println((h, f, l, t, budget_hf, budget_sofar))

In [11]:
# if (budget - budget_sofar) >= min(h, f)
#     could_add_l = (budget - budget_sofar) >= h
#     could_add_t = (budget - budget_sofar) >= f
#     if could_add_l && !could_add_t
#         l = l+1
#     end
#     if !could_add_l && could_add_t
#         t = t+1
#     end
#     if could_add_l && could_add_t
#         if fpart(lraw) > fpart(traw)
#             l = l+1
#         else
#             t = t+1
#         end
#     end
# end

# budget_sofar = h * f + h * l + f * t
# println((h, f, l, t, budget_hf, budget_sofar))

## Testing Random Scheme A

In [12]:
f = random_scheme_a()
print(f)
f.h * f.f + f.h * f.l + f.f * f.t

Fighter("No Name [Scheme A]", 50, 86, 33, 47)

9992

In [13]:
rt = random_team(() -> rand_rename(random_scheme_a()), 1000)
fighters_to_df(rt)[1:10, :]

Row,name,h,f,l,t
,String,Int64,Int64,Int64,Int64
1,Triangle Delphina,44,78,39,62
2,Mallet Denali,17,54,54,151
3,Keyanna the Hammer,28,74,42,91
4,Xerox Joshawn,30,83,34,78
5,Bonnibelle the Large,15,36,100,221
6,Frozen Zohan,20,88,29,87
7,Likable Malaya,49,22,149,73
8,Peculiar Wallet,95,43,49,29
9,Italie the Nightingale,46,66,52,69


In [14]:
counters = random_team(() -> pick_best_rdmly(library, rt, 99), 100)
fighters_to_df(counters)[1:10, :]

Row,name,h,f,l,t
,String,Int64,Int64,Int64,Int64
1,Temiloluwa the Piano,11,98,18,89
2,Diamond Kiarah,46,47,87,81
3,Ancient Cat,80,96,12,14
4,Beautiful Donut,52,49,82,65
5,Andria the Elegant,89,90,11,11
6,Robinson the Dodecahedron,22,98,26,74
7,Uber Jayleah,11,95,71,86
8,Mallet Colette,11,91,23,96
9,Azal the Snake,82,99,10,10


In [15]:
rt = random_team(() -> rand_rename(random_scheme_a()), 100)

100-element Vector{Fighter}:
 Fighter("Primordial Balloon", 95, 30, 66, 29)
 Fighter("Aggressive Pig", 36, 92, 32, 60)
 Fighter("Manta Shukri", 16, 46, 70, 177)
 Fighter("Yuleidy the Elephant", 27, 57, 62, 119)
 Fighter("Warm Kangaroo", 82, 46, 56, 35)
 Fighter("Marvell the Penguin", 33, 19, 211, 126)
 Fighter("Breighlyn the Fiery", 16, 112, 23, 70)
 Fighter("Wonderous Pteranodon", 65, 34, 92, 53)
 Fighter("Doctor Samia", 31, 126, 21, 43)
 Fighter("Unknown Bagel", 86, 45, 54, 33)
 Fighter("Wynn the Sheltering", 101, 28, 64, 25)
 Fighter("Risky Wombat", 60, 19, 132, 49)
 Fighter("Airplane Amiliano", 117, 40, 38, 21)
 ⋮
 Fighter("Nomi the Baboon", 13, 79, 27, 109)
 Fighter("Neurotic Cheetah", 104, 40, 46, 26)
 Fighter("Frozen Anna", 43, 68, 50, 72)
 Fighter("Nerdy Mariya", 87, 54, 42, 30)
 Fighter("Lain the Maximal", 12, 62, 37, 142)
 Fighter("Xcecutive Hinami", 35, 87, 34, 66)
 Fighter("Violet Tiger", 67, 12, 131, 34)
 Fighter("Super Giant", 24, 37, 116, 171)
 Fighter("Boot Oluwadara", 

In [34]:
for f in rt
    v = eval_battle_list(f, counters)
    if v < 90
        println(f)
        println(v)
    end
end

Fighter("Primordial Balloon", 95, 30, 66, 29)
-30
Fighter("Aggressive Pig", 36, 92, 32, 60)
-24
Fighter("Manta Shukri", 16, 46, 70, 177)
-16
Fighter("Yuleidy the Elephant", 27, 57, 62, 119)
-2
Fighter("Warm Kangaroo", 82, 46, 56, 35)
-92
Fighter("Marvell the Penguin", 33, 19, 211, 126)
22
Fighter("Breighlyn the Fiery", 16, 112, 23, 70)
8
Fighter("Wonderous Pteranodon", 65, 34, 92, 53)
-44
Fighter("Doctor Samia", 31, 126, 21, 43)
2
Fighter("Unknown Bagel", 86, 45, 54, 33)
-92
Fighter("Wynn the Sheltering", 101, 28, 64, 25)
22
Fighter("Risky Wombat", 60, 19, 132, 49)
-22
Fighter("Airplane Amiliano", 117, 40, 38, 21)
22
Fighter("Diogo the Uber", 33, 51, 77, 113)
-6
Fighter("Damario the T-Rex", 14, 72, 35, 118)
-2
Fighter("Studious Helicopter", 47, 73, 43, 62)
-98
Fighter("Generous Aleksandar", 49, 23, 146, 74)
-30
Fighter("Triangle Jameson", 21, 84, 32, 90)
-6
Fighter("Monstrous Commander", 40, 114, 22, 40)
2
Fighter("Ox Abraxas", 29, 73, 42, 91)
-24
Fighter("Filemon the Rock", 34, 60, 61

In [35]:
for f in counters
    v = eval_battle_list(f, rt)
    if v < 90
        println(f)
        println(v)
    end
end

Fighter("Temiloluwa the Piano", 11, 98, 18, 89)
28
Fighter("Diamond Kiarah", 46, 47, 87, 81)
4
Fighter("Ancient Cat", 80, 96, 12, 14)
30
Fighter("Beautiful Donut", 52, 49, 82, 65)
4
Fighter("Andria the Elegant", 89, 90, 11, 11)
40
Fighter("Robinson the Dodecahedron", 22, 98, 26, 74)
16
Fighter("Uber Jayleah", 11, 95, 71, 86)
26
Fighter("Mallet Colette", 11, 91, 23, 96)
32
Fighter("Azal the Snake", 82, 99, 10, 10)
38
Fighter("Butler Isham", 99, 79, 10, 15)
34
Fighter("Dancing Dragon", 99, 16, 82, 18)
22
Fighter("Helicopter Eziyah", 48, 49, 73, 84)
12
Fighter("Aerys the Orangutan", 10, 95, 31, 92)
30
Fighter("Noble Knight", 10, 94, 97, 86)
24
Fighter("Cheese Aivy", 51, 47, 77, 78)
18
Fighter("Frozen Maylon", 48, 51, 67, 85)
12
Fighter("Gentle Ibiza", 99, 82, 10, 10)
44
Fighter("Egotistical Abimael", 11, 93, 12, 95)
30
Fighter("Vacuum Poet", 11, 96, 27, 90)
26
Fighter("Peculiar Jabes", 90, 87, 11, 13)
36
Fighter("Briseis the Pug", 13, 92, 61, 87)
24
Fighter("Love the Blueberry", 92, 86, 1

In [17]:
for f in rt
    v = eval_battle_list(f, counters)
    if v > 30
        println(f)
        println(v)
    end
end

Fighter("Super Giant", 24, 37, 116, 171)
34


In [18]:
evaluate_generator(random_scheme_a, 100, 20)

-0.42400000000000004

In [19]:
evaluate_generator(random_fighter, 100, 20)

-0.8640000000000001

In [20]:
evaluate_generator(() -> random_tournament_winner(1), 100, 20)

-0.8550000000000001

In [21]:
evaluate_generator(() -> random_tournament_winner(2), 100, 20)

-0.8480000000000001

In [22]:
evaluate_generator(() -> random_tournament_winner(3), 100, 20)

-0.8420000000000002

In [23]:
evaluate_generator(() -> random_tournament_winner(4), 100, 20)

-0.7959999999999999

## Team battles

In [24]:
team_a = random_team(() -> rand_rename(random_scheme_a()), 100)
team_r0 = random_team(() -> rand_rename(random_tournament_winner(0)), 100)
team_r1 = random_team(() -> rand_rename(random_tournament_winner(1)), 100)
team_r2 = random_team(() -> rand_rename(random_tournament_winner(2)), 100)


100-element Vector{Fighter}:
 Fighter("Shovel Zaydian", 25, 80, 25, 92)
 Fighter("Xquisite Cheetah", 96, 61, 17, 41)
 Fighter("Wolf Jimenna", 77, 53, 34, 62)
 Fighter("Bagel Elissa", 72, 45, 70, 38)
 Fighter("Kainoa the Bee", 50, 61, 69, 57)
 Fighter("Paper Sibella", 86, 14, 94, 50)
 Fighter("Scissors Yu", 52, 41, 90, 77)
 Fighter("Yisreal the Boot", 88, 43, 32, 79)
 Fighter("Balloon Karion", 89, 19, 91, 11)
 Fighter("Knight Aryus", 47, 73, 34, 68)
 Fighter("Luminous Samurai", 38, 73, 40, 78)
 Fighter("Kalany the Manic", 27, 76, 83, 75)
 Fighter("Aksel the Happy", 28, 88, 96, 55)
 ⋮
 Fighter("Rishank the Dynamic", 85, 26, 62, 96)
 Fighter("Aliah the Heron", 32, 83, 50, 69)
 Fighter("Frog Chantelle", 63, 97, 35, 17)
 Fighter("Seasoned Bear", 33, 93, 32, 63)
 Fighter("Deslyn the Monkey", 77, 73, 39, 18)
 Fighter("Saliha the Maximal", 53, 52, 65, 73)
 Fighter("Mantis Aliya", 46, 92, 63, 31)
 Fighter("Kateri the Student", 90, 62, 39, 14)
 Fighter("Wise Scissors", 56, 74, 57, 36)
 Fighter("

In [25]:
eval_team_battle(team_a, counters)

-1

In [26]:
eval_team_battle(team_a, team_r0)

1

In [27]:
eval_team_battle(team_a, team_r1)

-1

In [28]:
eval_team_battle(team_a, team_r2)

1

In [29]:
eval_team_battle(team_r0, team_a)

-1

In [30]:
eval_team_battle(team_r0, team_r1)

-1

In [31]:
eval_team_battle(team_r0, team_r2)

-1

In [32]:
as = counters
bs = team_a
a_i = 1
b_i = 1
print("         ")
println(as[a_i])
print("         ")
println(bs[b_i])
while (a_i <= length(as)) && (b_i <= length(bs))
    res = eval_battle(as[a_i], bs[b_i])
    if res == 1
        println(string(as[a_i].name, " won against ", bs[b_i].name))
        b_i = b_i + 1
        print("         ")
        println(bs[b_i])
    else
        if res == 0
            println(string(as[a_i].name, " tied against ", bs[b_i].name))
            b_i = b_i + 1
            print("         ")
            println(bs[b_i])
        else
            println(string(as[a_i].name, " lost against ", bs[b_i].name))
        end
        a_i = a_i + 1
        print("         ")
        println(as[a_i])
    end
end
a_out = (a_i > length(as))
b_out = (b_i > length(as))
if a_out
    if b_out
        return 0
    else
        return -1
    end
else
    return 1
end


         Fighter("Temiloluwa the Piano", 11, 98, 18, 89)
         Fighter("Westley the Pangolin", 109, 42, 41, 22)
Temiloluwa the Piano lost against Westley the Pangolin
         Fighter("Diamond Kiarah", 46, 47, 87, 81)
Diamond Kiarah won against Westley the Pangolin
         Fighter("Diamond Engineer", 27, 133, 20, 44)
Diamond Kiarah won against Diamond Engineer
         Fighter("Passive Harlee", 117, 25, 56, 20)
Diamond Kiarah won against Passive Harlee
         Fighter("Fiery Dodecahedron", 33, 57, 66, 104)
Diamond Kiarah lost against Fiery Dodecahedron
         Fighter("Ancient Cat", 80, 96, 12, 14)
Ancient Cat won against Fiery Dodecahedron
         Fighter("Athanasia the Luminous", 21, 40, 99, 177)
Ancient Cat won against Athanasia the Luminous
         Fighter("Nguyen the Balanced", 49, 67, 49, 64)
Ancient Cat won against Nguyen the Balanced
         Fighter("Denver the Spider", 42, 108, 22, 42)
Ancient Cat lost against Denver the Spider
         Fighter("Beautiful Donut", 52, 

LoadError: BoundsError: attempt to access 100-element Vector{Fighter} at index [101]

In [33]:
a_i, b_i

(58, 101)